# Amazon Fine Food Reviews Analysis

Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

## Loading Data

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [3]:
con=sqlite3.connect('database.sqlite')
filtered_data=pd.read_sql_query("""SELECT * FROM Reviews WHERE Score !=3""",con)
def partition(x):
    if x<3:
        return 0
    return 1
actualScore=filtered_data['Score']
positiveNegative=actualScore.map(partition)
filtered_data['Score']=positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


#  Exploratory Data Analysis

## Data Cleaning: Deduplication

Removing Duplicates::

In [4]:
display=pd.read_sql_query("""SELECT * FROM Reviews WHERE Score !=3 AND UserId="AR5J8UI46CURR" ORDER BY ProductId""",con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


Here many rows are duplicated speacially summary part
<br>
Below code drop duplicates

In [5]:
sorted_data=filtered_data.sort_values('ProductId',axis=0,ascending=True,inplace=False,kind='quicksort',na_position='last')

In [6]:
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False)#removing duplicates
final.shape

(364173, 10)

In [7]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

<b>Observation:-</b> It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [8]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND HelpfulnessNumerator>HelpfulnessDenominator
ORDER BY ProductID
""", con)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [9]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [10]:
print(final.shape)
final['Score'].value_counts()

(364171, 10)


1    307061
0     57110
Name: Score, dtype: int64

# Text Preprocessing: Stemming, stop-word removal and Lemmatization
We also need to remove some unnecessary tags and words: <br>
Remove html tags<br>
Remove any punctuations or limited set of special characters like , or . or # etc.<br>
Check if the word is made up of english letters and is not alpha-numeric<br>
Check if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)<br>
Convert the word to lowercase<br>
Remove Stopwords<br>
Finally Snowball Stemming the word<br>

In [11]:
import re
i=0;
for sent in final['Text'].values:#.values considers values only not labels
    if(len(re.findall('<.*?>',sent))):
        print("{0}th review".format(i+1))
        print(sent)
        break
    i+=1

7th review
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [12]:
stop=set(stopwords.words('english'))
sno=nltk.stem.SnowballStemmer('english')

def cleanhtml(sentence):
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr,' ',sentence)
    return cleantext
def cleanpunc(sentence):
    cleaned=re.sub(r'[?|!|\'|#]',r'',sentence)
    cleaned=re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))#here tasty and related words to this are stemmed/stored as tasti

{'couldn', 'above', "haven't", 'on', 'few', 'you', 'had', 'under', 'why', 'same', "you'll", 'mustn', 's', 'or', 'ourselves', "weren't", 'through', 'that', 'doing', 'm', 'by', 'because', 'down', 'can', 'up', 'a', 'whom', 'against', "you're", 'hadn', 'just', 'but', 'more', "hadn't", "you'd", "wouldn't", "you've", 'am', 'between', "shouldn't", 'being', "mightn't", 'mightn', 'i', 'we', 'itself', 'his', 'its', 'doesn', 'and', 'into', 'until', 'my', 'didn', 'it', 'very', 'me', 'will', 'ma', 'wouldn', 'of', 'if', 'such', 'out', "don't", 'during', 'our', 'some', "won't", 'did', 'how', "that'll", 'he', "shan't", 'an', 'don', 'wasn', 'no', 'your', 'where', 're', 'other', 'so', 'were', 'once', "hasn't", 'when', 'any', 'this', 'all', 'hasn', 'then', 'won', 't', "should've", 'her', 'has', 'is', 'those', 'ain', 'only', 'y', 'haven', 'with', 'about', 'too', "needn't", 'having', 'isn', 'be', 'hers', 'yourselves', 'further', "isn't", 'needn', 'they', 'weren', 'than', 'these', 'there', 'o', 'not', 'afte

In [14]:
if not os.path.isfile('final.sqlite'): 
    final_string=[]
    all_positive_words=[]
    all_negative_words=[]
    for i,sent in enumerate(tqdm(final['Text'].values)):
        filtered_sentence=[]
        sent=cleanhtml(sent)#cleans html tag function created in previous cell
        for w in sent.split():
            for cleaned_words in cleanpunc(w).split():#here punctuations are also get cleaned so extra space is created
                if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                    if(cleaned_words.lower() not in stop):
                        s=(sno.stem(cleaned_words.lower())).encode("utf-8")
                        filtered_sentence.append(s)
                        if(final['Score'].values)[i]==1:#collecting +ve reviews 
                            all_positive_words.append(s)
                        if(final['Score'].values)[i]==0:#collecting -ve reviews
                            all_negative_words.append(s)
        str1=b" ".join(filtered_sentence)
        final_string.append(str1)
        
final['CleanedText']=final_string
final['CleanedText']=final['CleanedText'].str.decode("utf-8")
conn=sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory=str
final.to_sql('Reviews',conn,schema=None,if_exists='replace',index=True,index_label=None,chunksize=None,dtype=None)
conn.close()

with open('positive_words.pkl','wb') as f:
    pickle.dump(all_positive_words,f)
with open('negative_words.pkl','wb') as f:
    pickle.dump(all_negative_words,f)

100%|█████████████████████████████████| 364171/364171 [14:17<00:00, 424.93it/s]


In [16]:
if os.path.isfile('final.sqlite'):
    conn = sqlite3.connect('final.sqlite')
    final = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 """, conn)
    conn.close()
else:
    print("Please run the above cell")

# Bag of Words (BoW)

In [17]:
count_vect=CountVectorizer()
final_counts=count_vect.fit_transform(final['CleanedText'].values)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (364171, 70740)
the number of unique words  70740


# Bi-Grams and n-Grams

In [19]:
with open('positive_words.pkl', 'rb') as f:
    all_positive_words = pickle.load(f)
with open('negative_words.pkl', 'rb') as f:
    all_negative_words = pickle.load(f)
    
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most Common Positive Words : ",freq_dist_positive.most_common(20))
print("Most Common Negative Words : ",freq_dist_negative.most_common(20))

Most Common Positive Words :  [(b'like', 139345), (b'tast', 128766), (b'good', 112234), (b'flavor', 109085), (b'love', 107235), (b'use', 103814), (b'great', 103777), (b'one', 96561), (b'product', 90941), (b'tri', 86722), (b'tea', 83367), (b'coffe', 78440), (b'make', 75049), (b'get', 72048), (b'food', 64330), (b'would', 55556), (b'time', 55166), (b'buy', 54146), (b'realli', 52681), (b'eat', 51873)]
Most Common Negative Words :  [(b'tast', 34450), (b'like', 32303), (b'product', 28111), (b'one', 20536), (b'flavor', 19234), (b'would', 17969), (b'tri', 17742), (b'use', 15264), (b'good', 14920), (b'coffe', 14583), (b'get', 13774), (b'buy', 13734), (b'order', 12855), (b'food', 12621), (b'dont', 11855), (b'tea', 11546), (b'even', 11079), (b'box', 10816), (b'amazon', 10060), (b'make', 9835)]


<b>Observation:-</b> From the above it can be seen that the most common positive and the negative words overlap for eg. 'like' could be used as 'not like' etc.<br> 
So, it is a good idea to consider pairs of consequent words (bi-grams) or q sequnce of n consecutive words (n-grams)

In [20]:
count_vect=CountVectorizer(ngram_range=(1,2))
final_bigram_counts=count_vect.fit_transform(final['CleanedText'].values)
print("the type of count vectorizer ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])#on increasing grams dimension increases, that's why as compared to previous results of 115k here results gone to 2.9 million

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (364171, 2902276)
the number of unique words including both unigrams and bigrams  2902276


# TF-IDF

In [21]:
tf_idf_vect=TfidfVectorizer(ngram_range=(1,2))
final_tf_idf=tf_idf_vect.fit_transform(final['CleanedText'].values)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (364171, 2902276)
the number of unique words including both unigrams and bigrams  2902276


In [22]:
features=tf_idf_vect.get_feature_names()#gets all feature names present
print("some sample features(unique words in the corpus)",features[100000:100010])#prints 10 features between 100000 to 100010

some sample features(unique words in the corpus) ['anxious order', 'anxious pill', 'anxious pop', 'anxious probabl', 'anxious product', 'anxious puls', 'anxious purchas', 'anxious read', 'anxious realiz', 'anxious receiv']


In [23]:
#Displaying top 25 features with there tf-idf
def top_tfidf_feats(row,features,top_n=25):
    topn_ids=np.argsort(row)[::-1][:top_n]
    top_feats=[(features[i],row[i])for i in topn_ids]
    df=pd.DataFrame(top_feats)
    df.columns=['feature','tfidf']
    return df

top_tfidf=top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [24]:
top_tfidf#displaying top 25 names

,feature,tfidf
0,read sendak,0.192657
1,incorpor love,0.192657
2,flimsi take,0.192657
3,version paperback,0.192657
4,page open,0.192657
5,rosi movi,0.192657
6,grew read,0.192657
7,movi incorpor,0.192657
8,paperback seem,0.192657
9,keep page,0.192657


# Word2Vec

In [26]:
# Train your own Word2Vec model using your own text corpus
i=0
list_of_sent=[]
for sent in final['CleanedText'].values:
    list_of_sent.append(sent.split())

In [27]:
print(final['CleanedText'].values[0])
print("*****************************************************************")
print(list_of_sent[0])

witti littl book make son laugh loud recit car drive along alway sing refrain hes learn whale india droop love new word book introduc silli classic book will bet son still abl recit memori colleg
*****************************************************************
['witti', 'littl', 'book', 'make', 'son', 'laugh', 'loud', 'recit', 'car', 'drive', 'along', 'alway', 'sing', 'refrain', 'hes', 'learn', 'whale', 'india', 'droop', 'love', 'new', 'word', 'book', 'introduc', 'silli', 'classic', 'book', 'will', 'bet', 'son', 'still', 'abl', 'recit', 'memori', 'colleg']


In [28]:
#This is Word2Vec converter
w2v_model=Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [29]:
w2v_words = list(w2v_model.wv.vocab)#it gives list of all the words we have
#consider .wv as it is
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  21761
sample words  ['witti', 'littl', 'book', 'make', 'son', 'laugh', 'loud', 'recit', 'car', 'drive', 'along', 'alway', 'sing', 'refrain', 'hes', 'learn', 'whale', 'india', 'droop', 'love', 'new', 'word', 'introduc', 'silli', 'classic', 'will', 'bet', 'still', 'abl', 'memori', 'colleg', 'grew', 'read', 'sendak', 'watch', 'realli', 'rosi', 'movi', 'incorpor', 'howev', 'miss', 'hard', 'cover', 'version', 'paperback', 'seem', 'kind', 'flimsi', 'take', 'two']


In [30]:
#Note:tasti is stemming word of tasty...
w2v_model.wv.most_similar('tasti')#this prints all similar words to given word in decreasing order..and similarity level is measured between 0 to 1

[('delici', 0.8112898468971252),
 ('yummi', 0.7981356382369995),
 ('tastey', 0.7343237400054932),
 ('satisfi', 0.6929919123649597),
 ('nutriti', 0.6858402490615845),
 ('good', 0.6812770366668701),
 ('nice', 0.665293276309967),
 ('hearti', 0.6613863110542297),
 ('terrif', 0.6413846611976624),
 ('crunchi', 0.6332703232765198)]

In [31]:
w2v_model.wv.most_similar('like')

[('weird', 0.730994462966919),
 ('dislik', 0.6865944266319275),
 ('okay', 0.6859008073806763),
 ('resembl', 0.6665248870849609),
 ('except', 0.6475203633308411),
 ('gross', 0.6475184559822083),
 ('appeal', 0.6473760604858398),
 ('funki', 0.6422266364097595),
 ('prefer', 0.6403094530105591),
 ('hate', 0.6326681971549988)]

# Avg W2V, TFIDF-W2V

In [32]:
#Avg W2V
sent_vectors=[]
for sent in tqdm(list_of_sent):
    sent_vec=np.zeros(50)
    cnt_words=0
    for word in sent:
        if word in w2v_words:
            vec =w2v_model.wv[word]
            sent_vec+=vec
            cnt_words+=1
    if cnt_words!=0:
        sent_vec/=cnt_words#Avg W2V calculation
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

100%|█████████████████████████████████| 364171/364171 [28:37<00:00, 211.99it/s]


364171
50


In [33]:
model=TfidfVectorizer()
tf_idf_matrix=model.fit_transform(final['CleanedText'].values)
#converting a dictionary with word as a key, and the idf as a value
dictionary=dict(zip(model.get_feature_names(),list(model.idf_)))#model.idf_ calculates idf of each review

In [34]:
#TF-IDF Weighted W2V
tfidf_feat=model.get_feature_names()
tfidf_sent_vectors=[]
row=0
for sent in tqdm(list_of_sent):
    sent_vec=np.zeros(50)
    weight_sum=0
    for word in sent:
        if word in w2v_words:
            vec=w2v_model.wv[word]
            #tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            #to reduce the computation we have considered 
            #dictionary[word] = idf value of word in whole corpus
            #sent.count(word) = tf values of word in this review
            tf_idf=dictionary[word]*(sent.count(word)/len(sent))
            tf_idf=dictionary[word]*(sent.count(word)/len(sent))
            sent_vec+=(vec*tf_idf)
            weight_sum+=tf_idf
    if weight_sum!=0:
        sent_vec/=weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|█████████████████████████████████| 364171/364171 [35:41<00:00, 170.05it/s]
